**The code below to create the 4x4 mapping should create a positive mapping**

In [1]:
# This code was created by Darshini Rajamani of Purdue University

import random;

def get_random_number():
    return random.randint(-9, 9)

def is_valid(matrix, rows, cols, row, col, num):
    # initial assigment to check validation
    matrix[row][col] = num

    if row >= 2 and col == cols - 1:  # checking validatation atleast after filling two rows
        for i in range(cols):
            # check if x + w < 0
            if matrix[0][i] is not None and matrix[3][i] is not None:
                if matrix[0][i] + matrix[3][i] < 0:
                    matrix[row][col] = None  # Undo the initial assignment
                    return False

            # check if y + z < 0
            if matrix[1][i] is not None and matrix[2][i] is not None:
                if matrix[1][i] + matrix[2][i] < 0:
                    matrix[row][col] = None  # Undo the initial assignment
                    return False

    # Check row condition
    if col == 3:
        if sum(matrix[row][:2]) != num + matrix[row][2]:
            return False

    # Check column condition
    if row == 3:
        if sum([matrix[i][col] for i in range(2)]) != num + matrix[2][col]:
            return False

    # Check diagonal condition
    if row == 3 and col == 3:
        if matrix[0][3] + matrix[3][0] != matrix[1][2] + matrix[2][1]:
            return False

    # Check for row uniqueness
    if col == 3:
        matrix[row][col] = num
        if tuple(matrix[row]) in set(map(tuple, matrix[:row])):
            matrix[row][col] = None
            return False
        matrix[row][col] = None

    # Check that only one digit can be 0 in a row (to prevent 0,0,0,0) in a row)
    if col == 3:
        zero_count = sum([1 for x in matrix[row] if x == 0])
        if zero_count > 1:
            return False

    if all(x is not None for row in matrix for x in row):
        for i in range(cols):
            if matrix[1][i] + matrix[2][i] != matrix[0][i] + matrix[3][i]:
                return False

    matrix[row][col] = None
    return True

def create_matrix(rows, cols):
    matrix = [[None for _ in range(cols)] for _ in range(rows)]

    backtrackFailedAttempts = 0
    def backtrack(row, col):  # to check for complete and valid solution
        nonlocal backtrackFailedAttempts
        if backtrackFailedAttempts > 1000:
            return False

        if row == rows:
            return True

        nums = [i for i in range(-9, 10)]  # range [-9, 10)
        random.shuffle(nums)

        for num in nums:
            if is_valid(matrix, rows, cols, row, col, num):
                matrix[row][col] = num

                next_row = row
                next_col = col + 1
                if next_col == cols:
                    next_row += 1
                    next_col = 0

                if backtrack(next_row, next_col):
                    return True

        matrix[row][col] = None
        backtrackFailedAttempts = backtrackFailedAttempts + 1
        return False

    backtrack(0, 0)  # recursion

    return matrix



In [2]:
from scipy.optimize import linprog
import numpy as np

numberOfMappingsToCreate = 4000
#######################
### Create Mappings ###
#######################
listOfMappings = []
for i in range(numberOfMappingsToCreate):
  rows = 4
  cols = 4

  isAValidMatrix = False

  while not isAValidMatrix:
    isAValidMatrix = True
    matrix = create_matrix(rows, cols)
    for row in matrix:
      for element in row:
        if element is None:
          isAValidMatrix = False

  matrix[1], matrix[3] = matrix[3], matrix[1]

  listOfMappings.append(matrix)


###########################
# Check for Extendability #
###########################
extendableMappings = []
nonExtendableMappings = []

for matrix in listOfMappings:
  mat = [1, 1, 1, 1]
  for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]
  solve_for = [[min(matrix[0][2],matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]
  for_A_eq = [[1,-1,-1,1]]
  for_b_eq = [0]

  result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
  for row in matrix:
    print(row)
  if (result.success):
    extendableMappings.append(matrix)
    print("Extendable")
  else:
    nonExtendableMappings.append(matrix)
    print("Not extendable")
  print()

# Save the sample data to a file
np.save('/content/extendableMappings.npy', extendableMappings)
np.save('/content/nonExtendableMappings.npy', nonExtendableMappings)

Streaming output truncated to the last 5000 lines.
Not extendable

[0, 8, 2, 6]
[1, 3, 1, 3]
[8, 4, 4, 8]
[9, -1, 3, 5]
Extendable

[1, -2, 4, -5]
[9, 5, 7, 7]
[-1, -1, 2, -4]
[7, 6, 5, 8]
Not extendable

[6, 1, 8, -1]
[-4, 2, -4, 2]
[7, 0, 7, 0]
[-3, 1, -5, 3]
Not extendable

[-1, 6, 7, -2]
[6, 9, 6, 9]
[2, -1, 6, -5]
[9, 2, 5, 6]
Not extendable

[5, -1, 2, 2]
[2, 2, 1, 3]
[2, 2, 3, 1]
[-1, 5, 2, 2]
Extendable

[8, 7, 8, 7]
[6, 3, 7, 2]
[1, 8, 6, 3]
[-1, 4, 5, -2]
Not extendable

[0, 9, 1, 8]
[2, 0, 4, -2]
[0, 8, -1, 9]
[2, -1, 2, -1]
Not extendable

[7, 4, 7, 4]
[-2, 5, -1, 4]
[6, 4, 6, 4]
[-3, 5, -2, 4]
Not extendable

[-4, 5, -1, 2]
[8, 6, 9, 5]
[-4, 2, -2, 0]
[8, 3, 8, 3]
Not extendable

[-1, -3, -2, -2]
[8, 8, 8, 8]
[-1, -2, -2, -1]
[8, 9, 8, 9]
Not extendable

[2, 7, 3, 6]
[6, 3, 0, 9]
[-2, 9, 6, 1]
[2, 5, 3, 4]
Not extendable

[7, 1, 3, 5]
[5, 0, 7, -2]
[6, 3, 3, 6]
[4, 2, 7, -1]
Not extendable

[2, 0, -4, 6]
[5, 7, 4, 8]
[5, 2, 1, 6]
[8, 9, 9, 8]
Not extendable

[3, 2, 0, 5]
[

In [3]:
import numpy as np
extendableMappings = np.load('/content/extendableMappings.npy')
nonExtendableMappings = np.load('/content/nonExtendableMappings.npy')

In [4]:
print(f"extendableMappings: {len(extendableMappings)}")
print(f"nonExtendableMappings: {len(nonExtendableMappings)}")

extendableMappings: 997
nonExtendableMappings: 3003


In [5]:
from scipy.optimize import linprog

# Author: Luke Luschwitz and Karim El-Sharkawy
# Last edit: 10/26/23 7:35pm

# link to scipy.optimize.linprog: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

# Reference images from Professor Sinclair
# https://cdn.discordapp.com/attachments/993798931777585153/1165299134165696522/image0.jpg?ex=65465863&is=6533e363&hm=6a59d0a154f6bfa745b5194ccad740a781b196437bcee525abc6a2cf0768779d&
# https://cdn.discordapp.com/attachments/993798931777585153/1165291718418112603/IMG_7540.jpg?ex=6546517b&is=6533dc7b&hm=01582d955a387eac71189654067dff37c5b3988983e7b475c129e27bc427c6ca&


# The points in P(2,2) that were mapped from E(2,2).
# Points are mapped by f: E(2,2) -> P(2,2)

# These are what x, y, z, and w represent
# x = [a1,b1,c1,d1]
# y = [a2,b2,c2,d2]
# z = [a3,b3,c3,d3]
# w = [a4,b4,c4,d4]

# This matrix we know is extendible
x = [-1,3,1,1] # x is a mapped point in P(2,2)
y = [0,1,1,0] # y is a mapped point in P(2,2)
z = [0,2,1,1] # z is a mapped point in P(2,2)
w = [1,0,1,0] # w is a mapped point in P(2,2)

# This matrix we know is not extendible
# x = [-1,3,1,1] # x is a mapped point in P(2,2)
# y = [0,1,0,1] # y is a mapped point in P(2,2)
# z = [0,2,2,0] # z is a mapped point in P(2,2)
# w = [1,0,1,0] # w is a mapped point in P(2,2)

# This matrix we know is extendible
# x = [0,0,0,0] # x is a mapped point in P(2,2)
# y = [0,0,0,0] # y is a mapped point in P(2,2)
# z = [0,0,0,0] # z is a mapped point in P(2,2)
# w = [0,0,0,0] # w is a mapped point in P(2,2)


# The matrix that we are trying to map to R4
matrix = [x,
          y,
          z,
          w]

# In the scipy.optimize.linprog documentation, mat is c: "The coefficients of the linear objective function to be minimized".
# On Prof. SinClair's blackboard, mat is t: the vector of variables t1,t2,t3,t4 that need to be minimized/maximized. In our case, the objective function does not matter, we only care whether the solution exists.
mat = [1, 1, 1, 1]

# for_A_ub represents the system of inequalities
for_A_ub = [[1,0,0,0],
            [0,1,0,0],
            [0,0,1,0],
            [0,0,0,1],
            [-1,0,0,0],
            [0,-1,0,0],
            [0,0,-1,0],
            [0,0,0,-1],]

# In the scipy.optimize.linprog documentation, solve_for is b_ub: "The inequality constraint vector. Each element represents an upper bound on the corresponding value of A_ub @ x."
# On Prof. SinClair's blackboard, this is r: [r1,r2,r3,r4,-l1,-l2,-l3,-l4].
solve_for = [[min(matrix[0][2],matrix[0][3])],
             [min(matrix[1][2], matrix[1][3])],
             [min(matrix[2][2], matrix[2][3])],
             [min(matrix[3][2], matrix[3][3])],
             [-max(-matrix[0][0], -matrix[0][1])],
             [-max(-matrix[1][0], -matrix[1][1])],
             [-max(-matrix[2][0], -matrix[2][1])],
             [-max(-matrix[3][0], -matrix[3][1])]]

# In the scipy.optimize.linprog documentation, for_A_eq is A_eq: "The equality constraint matrix. Each row of A_eq specifies the coefficients of a linear equality constraint on x."
# On Prof. SinClair's blackboard, this is t1-t2-t3+t4=0.
for_A_eq = [[1,-1,-1,1]]

# In the scipy.optimize.linprog documentation, for_b_eq is b_eq: "The equality constraint vector. Each element of A_eq @ x must equal the corresponding element of b_eq."
# On Prof. SinClair's blackboard, this is t1-t2-t3+t4=0.
for_b_eq = [0]


# Solve for inequalities with scipy.optimize.linprog
result = linprog(c=mat, A_ub = for_A_ub, b_ub = solve_for, A_eq = for_A_eq, b_eq = for_b_eq, bounds = None)
# for how this function works, it has it such that A_ub <= b_ub at whatever c is equal to

# Print whether the inequality solving was successful (output of the linprog function)
# for row in matrix:
#     print(row)
# print(result)


In [6]:
import numpy as np

# This function returns the matrix B that is farthest away from the matrices in setA
def farthestBFromA(setB, setA):
  totalDistancesFromA = []
  # iterate through each B matrix
  for B in setB:
    currentBTotalDistance = 0
    # accumulate sum of distances from A to B (distance squared)
    for A in setA:
      currentBTotalDistance = currentBTotalDistance + (np.linalg.norm(np.subtract(B, A)) ** 2)
    totalDistancesFromA.append(currentBTotalDistance)

  orderedBsByDistance = []
  for i in range(len(totalDistancesFromA)):
    index = np.argmax(totalDistancesFromA)
    orderedBsByDistance.append(setB[index])
    totalDistancesFromA[index] = -np.inf
  return orderedBsByDistance



farthestBs = farthestBFromA(nonExtendableMappings, extendableMappings)

for i in range(len(farthestBs[0:3])):
  for row in farthestBs[i]:
      print(row)
  print(f"{i}th farthest from the extendables")
  print()

[8 9 9 8]
[-6 -8 -6 -8]
[7 9 8 8]
[-7 -8 -7 -8]
0th farthest from the extendables

[8 8 9 7]
[-8 -6 -8 -6]
[9 7 9 7]
[-7 -7 -8 -6]
1th farthest from the extendables

[9 8 9 8]
[-7 -7 -8 -6]
[9 9 9 9]
[-7 -6 -8 -5]
2th farthest from the extendables



In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score

# Generate input by adding the farthestB matrix to the list of extendableMappings
features = np.array(np.concatenate((extendableMappings, np.array([farthestBs[0]]))))
print(f"len(extendableMappings): {len(extendableMappings)}")
print(f"len(features): {len(features)}")

# Flatten the matrices into 1D arrays
features = features.reshape(len(features), -1)

# Generate labels (0 or 1)
labels = [0]*len(extendableMappings) + [1]
print(f"len(labels): {len(labels)}")
print(f"labels: {labels}")

# Create and train a logistic regression model
model = svm.SVC(kernel='linear', C=1, coef0=0.0)
model.fit(features, labels)

model.intercept_ = [0.0]

# Make predictions on the test set
predictions = model.predict(features)

# Evaluate the accuracy of the model
accuracy = accuracy_score(labels, predictions)
print(f"Accuracy: {accuracy}")

# Get the coefficients (weights) of the hyperplane
coefficients = model.coef_.reshape(4, 4)

# Intercept of the hyperplane
intercept = model.intercept_

print("Coefficients:\n", coefficients)
print("Intercept:", intercept)


len(extendableMappings): 997
len(features): 998
len(labels): 998
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
classifier = coefficients * 1000
classifier = np.round(classifier)
print("classifier")
print(classifier)
print("coefficients")
print(coefficients)

allMappings = np.concatenate((extendableMappings, nonExtendableMappings), axis=0)
print(f"len(allMappings): {len(allMappings)}")
print()

totalMappingsInClass1 = 0
totalMappingsInClass2 = 0
classifiedMappingResults = []

for i in range(len(allMappings)):
  dotProduct = np.dot(classifier, allMappings[i])
  dotProduct = np.sum(dotProduct)
  extendableOrNot = "E" if i<len(extendableMappings) else "N"
  if dotProduct < 0:
    print(f"{extendableOrNot} {dotProduct} (class 1)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 1))
    totalMappingsInClass1 += 1
  else:
    print(f"{extendableOrNot} {dotProduct} (2)")
    classifiedMappingResults.append((allMappings[i], extendableOrNot, 1))
    totalMappingsInClass2 += 1

for mapping, extendability, _class in classifiedMappingResults:
  if (extendability == "E" and _class == 2):
    print("Bad classifier, extendableMapping in class 2")
    print(mapping, extendability, _class)
    break
  if totalMappingsInClass2 < 1:
    print("Bad classifier, no mappings in class 2")
    break

print(f"totalMappingsInClass1 {totalMappingsInClass1}")
print(f"totalMappingsInClass2 {totalMappingsInClass2}")

classifier
[[  4.   7.   7.   4.]
 [-22. -33. -26. -29.]
 [ -4.   7.   7.  -4.]
 [-29. -33. -26. -37.]]
coefficients
[[ 0.00367647  0.00735294  0.00735294  0.00367647]
 [-0.02205882 -0.03308824 -0.02573529 -0.02941176]
 [-0.00367647  0.00735294  0.00735294 -0.00367647]
 [-0.02941176 -0.03308824 -0.02573529 -0.03676471]]
len(allMappings): 4000

E -4078.0 (class 1)
E -3074.0 (class 1)
E -3726.0 (class 1)
E -3672.0 (class 1)
E -4442.0 (class 1)
E -3316.0 (class 1)
E -5004.0 (class 1)
E -4318.0 (class 1)
E -4140.0 (class 1)
E -4428.0 (class 1)
E -5856.0 (class 1)
E -3408.0 (class 1)
E -5502.0 (class 1)
E -4822.0 (class 1)
E -5004.0 (class 1)
E -5000.0 (class 1)
E -2982.0 (class 1)
E -4520.0 (class 1)
E -4022.0 (class 1)
E -4968.0 (class 1)
E -3190.0 (class 1)
E -4760.0 (class 1)
E -3878.0 (class 1)
E -4372.0 (class 1)
E -3814.0 (class 1)
E -4460.0 (class 1)
E -5182.0 (class 1)
E -4056.0 (class 1)
E -3788.0 (class 1)
E -4466.0 (class 1)
E -4498.0 (class 1)
E -5032.0 (class 1)
E -4756.0 (cla